In [ ]:
### imports

# external modules
import matplotlib.pyplot as plt
import importlib

# local modules
import get_oms_data
importlib.reload(get_oms_data)
from get_oms_data import get_oms_api, get_oms_data, get_oms_response_attribute

In [ ]:
# get the omsapi instance

omsapi = get_oms_api()

In [ ]:
# example: get run information for a single run

runnb = 297050
run_info = get_oms_data( omsapi, 'runs', runnb )
print(run_info)

In [ ]:
# example: get only specific info for a single run

runnb = 297050
run_info_spec = get_oms_data( omsapi, 'runs', runnb, attributes=['l1_rate','l1_menu'] )
print(run_info_spec)
print(run_info_spec['data'])
print(run_info_spec['data'][0]['attributes'])

In [ ]:
# example: get specific run information for multiple runs

runnb = (297050,297056)
run_info_mult = get_oms_data( omsapi, 'runs', runnb, attributes=['l1_rate','l1_menu'] )
print(run_info_mult)
l1_rate = get_oms_response_attribute( run_info_mult, 'l1_rate' )
print(l1_rate)

In [ ]:
# example: get lumisection information

runnb = 297050
ls_info = get_oms_data( omsapi, 'lumisections', runnb, attributes=['lumisection_number','pileup','delivered_lumi','recorded_lumi'] )
#print(ls_info)
ls_nbs = get_oms_response_attribute(ls_info,'lumisection_number')
pileup = get_oms_response_attribute(ls_info,'pileup')
del_lumi = get_oms_response_attribute(ls_info,'delivered_lumi')
rec_lumi = get_oms_response_attribute(ls_info,'recorded_lumi')

plt.figure()
plt.plot(ls_nbs,pileup)
plt.figure()
plt.plot(ls_nbs,del_lumi,color='b')
plt.plot(ls_nbs,rec_lumi,color='g')